In [179]:

import pkg_resources
import subprocess
import re
import json

package = 'openai'
try:
    dist = pkg_resources.get_distribution(package)
    print(f"{package} is already installed (version {dist.version}).")
except pkg_resources.DistributionNotFound:
    print(f"{package} is not installed. Installing...")
    subprocess.call(['pip3', 'install', package])
    print(f"{package} has been installed.")

openai is already installed (version 0.27.8).


In [2]:
#https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
#https://www.youtube.com/watch?v=uCKH8bmPgFs&t=502s

In [2]:
import openai
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = "https://gpt4resource.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "618bf90290544e54b47bcfc2dba743da"

system_message = 'Assistant is an intelligent chatbot designed to help users create programming training materials'
user_message1 = 'Could you please provide me with incorrect code in Node.js, followed by a corresponding unit test that will fail due to the introduced bug? At the end provide the corrected code. Area of interest is web development.'
response = openai.ChatCompletion.create(
    #gpt-35-turbo max tokens = 4,096
    engine="testdeploy", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message1}
    ],
    temperature = 0 # This is the degree of randomness of the model's output
)

In [51]:
# print(response)
# print(50*'-')

output = response['choices'][0]['message']['content']

print('output: ', output)
print(50*'-')
print('tokens: ', response['usage'])

In [160]:
def extract_language(text):
    pattern = r"```(.*)\n"
    matches = re.findall(pattern, text)
    languages = [match for match in matches if match != '']
    return languages

def extract_filename(text):
    pattern = r"// (.*)\n"
    matches = re.findall(pattern, text)
    return matches

def remove_before_second_newline(text):
    lines = text.split('\n')
    new_text = '\n'.join(lines[2:])
    return new_text[1:]

def extract_code(text):
    pattern = r"```([\s\S]*?)```"
    matches = re.findall(pattern, text)
    code = []
    for match in matches:
        code.append(remove_before_second_newline(match))
    return code

In [169]:
languages = extract_language(output)
filenames = extract_filename(output)
code = extract_code(output)

In [170]:
languages

['javascript', 'javascript', 'javascript']

In [171]:
filenames

['server.js', 'server.test.js', 'server.js']

In [172]:
code

["const express = require('express');\nconst app = express();\n\napp.get('/', (req, res) => {\n  res.send('Hello World!');\n});\n\napp.listen(3000, () => {\n  console.log('Server started on port 3000');\n});\n",
 "const request = require('supertest');\nconst app = require('./server');\n\ndescribe('GET /', () => {\n  it('should return 200 OK', (done) => {\n    request(app)\n      .get('/')\n      .expect(200, done);\n  });\n});\n\ndescribe('Server', () => {\n  it('should start without errors', (done) => {\n    const server = app.listen(3000, () => {\n      server.close();\n      done();\n    });\n  });\n});\n",
 "const express = require('express');\nconst app = express();\n\napp.get('/', (req, res) => {\n  res.send('Hello World!');\n});\n\nconst server = app.listen(3000, () => {\n  console.log('Server started on port 3000');\n});\n\nserver.on('error', (err) => {\n  console.error(`Server error: ${err}`);\n});\n"]

In [177]:
data = {
    'labels': ['Incorrect code', 'Unit test', 'Corrected code'],
    'languages': languages,
    'filenames': filenames,
    'code': code
}

json_filename = 'label_lang_file_code.json'
with open(json_filename, 'w') as file:
    json.dump(data, file)

In [28]:
#RUN CELL TO SAVE OUTPUT IN A TXT FILE
import os

folder_path = 'txt_outputs'
text_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
txt_number_list = []
for file_name in text_files:
    txt_number = file_name[-5]
    txt_number = int(txt_number)
    txt_number_list.append(txt_number)
if not txt_number_list: #check if list is empty, i.e. no txt files yet
    new_txt_number = str(1)
else:
    new_txt_number = str(max(txt_number_list) + 1)
text_file = open('txt_outputs/output' + '_' + new_txt_number + '.txt', 'w')
text_file.write(system_message + '\n')
text_file.write(50*'-' + '\n')
text_file.write(user_message1 + '\n')
text_file.write(50*'-' + '\n')
text_file.write(output + '\n')
text_file.close()